In [ ]:
import numpy as np
import pandas as pd
import sklearn.linear_model import LinearRegression

# Assuming 'df' is your DataFrame containing the data
df['log_academic_success'] = np.log(df['academic_success'])
df['log_study_hours'] = np.log(df['study_hours'])
df['log_attendance'] = np.log(df['attendance'])
# Transform other relevant factors similarly


In [ ]:
df.head()

In [ ]:
df.dropna(inplace=True)

In [ ]:
#Log-Log Regression using Scikit-Learn

# Example data
x = np.array([1, 2, 3, 4, 5])
y = np.array([2, 4, 6, 8, 10])

# Log transform the data
log_x = np.log(x).reshape(-1, 1)
log_y = np.log(y)

# Fit the linear regression model
model = LinearRegression()
model.fit(log_x, log_y)

# Coefficients
slope = model.coef_[0]
intercept = model.intercept_

print(f"Slope: {slope}, Intercept: {intercept}")


In [ ]:
from sklearn.linear_model import LinearRegression

#fit linear regression model to transfored data
# Selecting the transformed variables
X = df[['log_study_hours', 'log_attendance']]  # Add other log-transformed factors
y = df['log_academic_success']

# Fitting the model
model = LinearRegression()
model.fit(X, y)

# Getting the coefficients
coefficients = model.coef_
intercept = model.intercept_

print(f"Intercept: {intercept}, Coefficients: {coefficients}")
